In [10]:
# Importando as bibliotecas
import bs4 
from bs4 import BeautifulSoup
import urllib.request as urllib_request
from urllib.request import Request, urlopen, urlretrieve
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
import pandas as pd

print("BeautifulSoup ->", bs4.__version__)
print("urllib ->", urllib_request.__version__)
print("pandas ->", pd.__version__)

BeautifulSoup -> 4.9.3
urllib -> 3.8
pandas -> 1.2.4


In [11]:
#Declarando variável cards
cards = []
garagem = []
quartos = []
rua = []
bairro = []

##Obtendo HTML e o total de paginas
pages = int(400)


##interando por todas as paginas do site
for i in range(pages):
    ##Obtendo o HTML
   
    response = 'https://www.zapimoveis.com.br/venda/casas/?pagina=' + str(i) + '&transacao=Venda&tipoUnidade=Residencial,Casa&tipo=Im%C3%B3vel%20usado'
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    requisicao = Request(response,headers = headers)

    try:
        response = urlopen(requisicao)
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.status(), e.reason())
    except URLError as e:
        print(e.reason()) 

    
    #Obtendo as Tags de interesse
    anuncios = soup.find('div',{'class':'listings__container'}).findAll('div',class_='card-listing simple-card')

    #Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Valor
        valor = anuncio.find('p', {'class': 'simple-card__price js-price color-darker heading-regular heading-regular__bolder align-left'})
        if valor == None:
            card['valor'] = int('0')
        else :
            card['valor'] = int(valor.get_text().replace('R$','').replace('\n','').replace('.',''))
            
        
        #descricao
        descricao = anuncio.find('span',{'class':'simple-card__text text-regular'})
        if descricao == None:
            card['descricao'] = str('Vazio')
        else :
            card['descricao'] = str(descricao.get_text().replace('-',' ').replace('\n',''))


        #endereco
        endereco = anuncio.find('h2',{'class':'simple-card__address color-dark text-regular'})
        if endereco == None:
            card['local'], card['bairro/cidade'] = str('Não disponivel', 'Não disponivel')
        else :
            card['local'], card['bairro/cidade'], *outro = endereco.get_text().replace('\n','').replace('  ','').split(',')

            
        #quartos
        quartos = anuncio.find('span',{'itemprop':'numberOfRooms'})
        if quartos == None:
            card['quartos'] = int(0)
        else :
            card['quartos'] = int(quartos.get_text().replace(' ','').replace('\n',''))
        

        
        #vagas
        garagem = anuncio.find('li',{'class':'feature__item text-small js-parking-spaces'})
        if garagem == None:
            card['garagem'] = int(0)
        else :
            card['garagem'] = int(garagem.get_text().replace(' ','').replace('\n',''))

        
            
        #banheiros
        banheiro = anuncio.find('span',{'itemprop':'numberOfBathroomsTotal'})
        if banheiro == None:
            card['banheiro'] = int(0)
        else :
            card['banheiro'] = int(banheiro.get_text().replace(' ','').replace('\n',''))

          
        #metragem
        metro = anuncio.find('span',{'itemprop':'floorSize'})
        if metro == None:
            card['area(m2)'] = int(0)
        else :
            card['area(m2)'] = int(metro.get_text().replace(' ','').replace('\n','').replace('m²',''))


            
        #IPTU
        iptu = anuncio.find('li',{'class':'card-price__item iptu text-regular'})
        if iptu == None:
            card['iptu'] = int('0')
        else :
            card['iptu'] = int(iptu.get_text().replace('R$','').replace('\n','').replace('.','').replace('IPTU','').replace(' ',''))

                          
            
        #condominio
        condominio = anuncio.find('li',{'class':'card-price__item condominium text-regular'})
        if condominio == None:
            card['condominio'] = int('0')
        else :
            card['condominio'] = int(condominio.get_text().replace('condomínioR$','').replace(' ','').replace('.',''))
        
        
        #tipo
        tipo = anuncio.find('small')      
        if tipo == None:
            card['novidade'] = str('Nenhuma')
        else :
            card['novidade'] = str(tipo.get_text())


            
        cards.append(card)


        
dataset = pd.DataFrame(cards)
dataset.to_csv('./dados/dataset-casas-geral.csv', sep=';' , index=False, encoding='utf-8-sig')
dataset

,valor,descricao,local,bairro/cidade,quartos,garagem,banheiro,area(m2),iptu,condominio,novidade
0,652000,Excelente casa de alto padrão em ótima localiz...,Centro II,Brusque,2,2,1,165,0,0,Super destaque
1,8000000,"Cidade Jardim nobre, casa de alto padrao em ru...",Rua Doutor Alcides da Silveira Faro,Cidade Jardim,4,3,6,600,0,0,Super destaque
2,485000,"Sobrado composto por 3 dormitórios, 1 suíte, s...",Rua Waldemar Mancini,Itaquera,3,2,2,102,0,0,Super destaque
3,450000,"A casa são 3 quartos, dois banheiros sendo um ...",Santos Dumont,Vila Velha,3,3,2,180,0,0,Destaque
4,350000,Casa com 176mts Lote 10x30 Casa com 3 quartos...,Setor Sul Jamil Miguel,Anápolis,3,2,3,176,0,0,Destaque
...,...,...,...,...,...,...,...,...,...,...,...
9595,640000,"Excelente sobrado, muito espaçoso e com ótima ...",Rua Maria Roque,Imirim,3,2,2,150,140,0,Nenhuma
9596,300000,"Casa com 3 dormitórios à venda, 100 m² por R$ ...",Residencial Jardins do Lago \t,Anápolis,3,3,1,100,0,0,Nenhuma
9597,480000,"Casa com 3 dormitórios à venda, 300 m² por R$ ...",Residencial Portal do Cerrado,Anápolis,3,0,2,300,0,0,Nenhuma
9598,370000,"Casa com edícula em Caraguatatuba, na praia de...",Jardim do Sol,Caraguatatuba,3,3,3,175,0,0,Nenhuma


In [16]:
#Declarando variável cards
cards = []
garagem = []
quartos = []
rua = []
bairro = []

##Obtendo HTML e o total de paginas
pages = int(350)


##interando por todas as paginas do site
for i in range(pages):
    ##Obtendo o HTML
   
    response = 'https://www.zapimoveis.com.br/venda/apartamentos/?pagina='+str(i)
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    requisicao = Request(response,headers = headers)

    try:
        response = urlopen(requisicao)
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.status(), e.reason())
    except URLError as e:
        print(e.reason()) 

    
    #Obtendo as Tags de interesse
    anuncios = soup.find('div',{'class':'listings__container'}).findAll('div',class_='card-listing simple-card')

    #Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Valor
        valor = anuncio.find('p', {'class': 'simple-card__price js-price color-darker heading-regular heading-regular__bolder align-left'})
        if valor == None:
            card['valor'] = int('0')
        else :
            card['valor'] = int(valor.get_text().replace('R$','').replace('\n','').replace('.',''))
            
        
        #descricao
        descricao = anuncio.find('span',{'class':'simple-card__text text-regular'})
        if descricao == None:
            card['descricao'] = str('Vazio')
        else :
            card['descricao'] = str(descricao.get_text().replace('-',' ').replace('\n',''))


        #endereco
        endereco = anuncio.find('h2',{'class':'simple-card__address color-dark text-regular'})
        if endereco == None:
            card['local'], card['bairro/cidade'] = 'Não disponivel', 'Não disponivel'
        else :
            card['local'], card['bairro/cidade'], *outro = endereco.get_text().replace('\n','').replace('  ','').split(',')

            
        #quartos
        quartos = anuncio.find('span',{'itemprop':'numberOfRooms'})
        if quartos == None:
            card['quartos'] = int(0)
        else :
            card['quartos'] = int(quartos.get_text().replace(' ','').replace('\n',''))
        

        
        #vagas
        garagem = anuncio.find('li',{'class':'feature__item text-small js-parking-spaces'})
        if garagem == None:
            card['garagem'] = int(0)
        else :
            card['garagem'] = int(garagem.get_text().replace(' ','').replace('\n',''))


        #banheiros
        banheiro = anuncio.find('span',{'itemprop':'numberOfBathroomsTotal'})
        if banheiro == None:
            card['banheiro'] = int(0)
        else :
            card['banheiro'] = int(banheiro.get_text().replace(' ','').replace('\n',''))

            
        #metragem
        metro = anuncio.find('span',{'itemprop':'floorSize'})
        if metro == None:
            card['area(m2)'] = int(0)
        else :
            card['area(m2)'] = int(metro.get_text().replace(' ','').replace('\n','').replace('m²',''))


        #IPTU
        iptu = anuncio.find('li',{'class':'card-price__item iptu text-regular'})
        if iptu == None:
            card['iptu'] = int('0')
        else :
            card['iptu'] = int(iptu.get_text().replace('R$','').replace('\n','').replace('.','').replace('IPTU','').replace(' ',''))


        #condominio
        condominio = anuncio.find('li',{'class':'card-price__item condominium text-regular'})
        if condominio == None:
            card['condominio'] = int('0')
        else :
            card['condominio'] = int(condominio.get_text().replace('condomínioR$','').replace(' ','').replace('.',''))
        
        #tipo
        tipo = anuncio.find('small')      
        if tipo == None:
            card['novidade'] = str('Nenhuma')
        else :
            card['novidade'] = str(tipo.get_text())
            
        cards.append(card)

        
dataset = pd.DataFrame(cards)
dataset.to_csv('./dados/dataset-ap-geral.csv', sep=';', index=False, encoding='utf-8-sig')
dataset

,valor,descricao,local,bairro/cidade,quartos,garagem,banheiro,area(m2),iptu,condominio,novidade
0,2390000,Encante se com essa esplendorosa cobertura dup...,Exposição,Caxias do Sul,3,4,4,345,0,800,Super destaque
1,720000,LIndo Apartamento no Bairro Santa Paula em São...,Rua Nilo Peçanha,Santa Paula,3,2,3,128,210,950,Super destaque
2,170000,Abrimos as portas para quem quer alugar rápido...,Praça Doutor Mário Margarido,Liberdade,1,0,1,33,0,364,Super destaque
3,352000,ADVANCED Melhor Condomínio do Bela Vista em Os...,Avenida Santo Antônio,Vila Osasco,2,1,1,52,30,350,Nenhuma
4,355000,Lindo apto todo decorado Com sala de visita em...,Tubalina,Uberlândia,2,2,2,68,0,344,Nenhuma
...,...,...,...,...,...,...,...,...,...,...,...
8395,361705,"Lindo apartamento de 2 dormitórios, sendo 1 su...",Rua Irmão Norberto Francisco Rauch,Jardim Carvalho,2,1,1,62,46,480,Nenhuma
8396,475100,Maravilhoso apartamento em prédio novo com laz...,José Menino,Santos,1,1,1,48,196,335,Destaque
8397,347300,"Ótimo apartamento com 70m² de área util, 3 dor...",Rua Cristiano Angeli,Assunção,3,1,1,70,90,390,Nenhuma
8398,320000,Excelente e espaçoso apartamento de 2 dormitór...,Avenida Cinco de Novembro,Teresópolis,2,1,1,74,0,380,Nenhuma


In [21]:
#Declarando variável cards
cards = []
garagem = []
quartos = []
rua = []
bairro = []

##Obtendo HTML e o total de paginas
pages = int(400)


##interando por todas as paginas do site
for i in range(pages):
    
    ##Obtendo o HTML
    response = 'https://www.zapimoveis.com.br/venda/casas/rj+rio-de-janeiro/?pagina='+str(i)
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    requisicao = Request(response,headers = headers)

    try:
        response = urlopen(requisicao)
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.status(), e.reason())
    except URLError as e:
        print(e.reason()) 

    
    #Obtendo as Tags de interesse
    anuncios = soup.find('div',{'class':'listings__container'}).findAll('div',class_='simple-card__box')

    #Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Valor
        valor = anuncio.find('p', {'class': 'simple-card__price js-price color-darker heading-regular heading-regular__bolder align-left'})
        if valor == None:
            card['valor'] = int('0')
        else :
            card['valor'] = int(valor.get_text().replace('R$','').replace('\n','').replace('.',''))
            
        
        #descricao
        descricao = anuncio.find('span',{'class':'simple-card__text text-regular'})
        if descricao == None:
            card['descricao'] = str('Vazio')
        else :
            card['descricao'] = str(descricao.get_text().replace('-',' ').replace('\n',''))


        #endereco
        endereco = anuncio.find('h2',{'class':'simple-card__address color-dark text-regular'})
        if endereco == None:
            card['local'], card['bairro/cidade'] = 'Não disponivel', 'Não disponivel'
        else :
            card['local'], card['bairro/cidade'], *outro = endereco.get_text().replace('\n','').replace('  ','').split(',')

            
        #quartos
        quartos = anuncio.find('span',{'itemprop':'numberOfRooms'})
        if quartos == None:
            card['quarto-min'], card['quarto-max'] = '0','0' 
        elif '-' in quartos.get_text() :
            card['quarto-min'], card['quarto-max'], *outro = quartos.get_text().replace(' ','').replace('\n','').split('-')
        else:        
           card['quarto-min'], card['quarto-max'] = quartos.get_text().replace(' ','').replace('\n',''),quartos.get_text().replace(' ','').replace('\n','')    
        
        
        #vagas
        garagem = anuncio.find('li',{'class':'feature__item text-small js-parking-spaces'})
        if garagem == None:
            card['garagem-min'], card['garagem-max'] = '0' , '0'
        elif '-' in garagem.get_text() :
            card['garagem-min'], card['garagem-max'] = garagem.get_text().replace(' ','').replace('\n','').split('-')
        else:    
            card['garagem-min'], card['garagem-max'] = garagem.get_text().replace(' ','').replace('\n','') , garagem.get_text().replace(' ','').replace('\n','')

        
            
        #banheiros
        banheiro = anuncio.find('span',{'itemprop':'numberOfBathroomsTotal'})
        if banheiro == None:
            card['banheiro-min'], card['banheiro-max'] = '0','0'
        elif '-' in banheiro.get_text() :
            card['banheiro-min'], card['banheiro-max'] = banheiro.get_text().replace(' ','').replace('\n','').split('-')
        else:    
            card['banheiro-min'], card['banheiro-max'] = banheiro.get_text().replace(' ','').replace('\n',''),banheiro.get_text().replace(' ','').replace('\n','')

          
        #metragem
        metro = anuncio.find('span',{'itemprop':'floorSize'})
        if metro == None:
            card['area(m2)-min'], card['area(m2)-max'] = '0','0'
        elif '-' in metro.get_text() :
            card['area(m2)-min'], card['area(m2)-max'], *outro = metro.get_text().replace(' ','').replace('\n','').replace('m²','').split('-')
        else:
            card['area(m2)-min'], card['area(m2)-max'] = metro.get_text().replace(' ','').replace('\n','').replace('m²',''),metro.get_text().replace(' ','').replace('\n','').replace('m²','')

            
        #IPTU
        iptu = anuncio.find('li',{'class':'card-price__item iptu text-regular'})
        if iptu == None:
            card['iptu'] = int('0')
        else :
            card['iptu'] = int(iptu.get_text().replace('R$','').replace('\n','').replace('.','').replace('IPTU','').replace(' ',''))

                          
            
        #condominio
        condominio = anuncio.find('li',{'class':'card-price__item condominium text-regular'})
        if condominio == None:
            card['condominio'] = int('0')
        else :
            card['condominio'] = int(condominio.get_text().replace('condomínioR$','').replace(' ','').replace('.',''))
        
        #tipo
        tipo = anuncio.find('small')      
        if tipo == None:
            card['novidade'] = str('Nenhuma')
        else :
            card['novidade'] = str(tipo.get_text())
            
        cards.append(card)

        
dataset = pd.DataFrame(cards)
dataset.to_csv('./dados/dataset-casas-rj.csv', sep=';', index=False, encoding='utf-8-sig')
dataset

,valor,descricao,local,bairro/cidade,quarto-min,quarto-max,garagem-min,garagem-max,banheiro-min,banheiro-max,area(m2)-min,area(m2)-max,iptu,condominio,novidade
0,6850000,Casa única no Alto Jardim Botânico! Uma casa ...,Rua Peri,Jardim Botânico,4,4,2,2,8,8,508,610,0,0,Nenhuma
1,1999000,O Condomínio Residencial Camboata está a pouco...,Rua Geógrafo Amora,Piratininga,3,5,3,4,4,6,400,700,0,0,Nenhuma
2,2990000,Mansões Rio Mar. Excelentes projetos de casas ...,Avenida das Américas,Barra da Tijuca,5,5,3,3,6,6,460,540,0,0,Nenhuma
3,4999998,Porto dos Cabritos – Barra da Tijuca – Lindíss...,Barra da Tijuca,Rio de Janeiro,5,5,4,4,5,5,950,950,0,0,Nenhuma
4,5990000,Terraço com piscina Sala em 4 ambientes Sa...,Rua Caio Mário,Gávea,6,6,7,7,10,10,733,733,1,1,Nenhuma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9595,140000,"Excelente apartamento tipo casa térreo, frente...",Rua Leopoldino Bastos,Engenho Novo,2,2,1,1,1,1,92,92,0,0,Nenhuma
9596,3900000,"Ótima casa de 3 andares, com 991m2 de área con...",Rua Henrique Cavaleiro,São Conrado,6,6,4,4,3,3,992,992,28102,1200,Nenhuma
9597,309900,"Em condomínio exclusivo, apenas 8 casas e a 80...",Rua Joaquim Tourinho,Jacarepaguá,2,2,1,1,2,2,80,80,60,200,Nenhuma
9598,2000000,Maravilhosa casa Triplex Com Terraço com 300 m...,Rua Flávio Cavalcanti,Recreio Dos Bandeirantes,3,3,3,3,5,5,300,300,8600,760,Nenhuma
